In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
folder = 'sp_matrix_tag_pop'
group_type = 'high'
group_dir_ = '../data/groups/' + group_type
group_info = 'group_info/' + group_type
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [2]:
df = pd.read_pickle(os.path.join(dir_, folder, group_info, file_name))
df[:5]

,uid,tid,count,tags,rating,pop,group_info_2,group_info_3,group_info_4,group_info_5
0,0,0,2,1,1.50,5,48.0,33.0,28.0,21.0
2,0,1,2,2,1.50,10,48.0,33.0,28.0,21.0
7,0,2,4,0,3.00,24,48.0,33.0,28.0,21.0
8,0,3,1,0,0.25,9,48.0,33.0,28.0,21.0
10,0,4,5,1,3.50,14,48.0,33.0,28.0,21.0


In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
group2 = {}
group3 = {}
group4 = {}
group5 = {}
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    j = 0;
    for group in groups:
        for user in group:
            if len(group) == 2:
                group2[user] = j
            elif len(group) == 3:
                group3[user] = j
            elif len(group) == 4:
                group4[user] = j
            elif len(group) == 5:
                group5[user] = j
        j += 1

In [4]:
predictions2 = pd.read_pickle(os.path.join(dir_, folder, group_info, '2_topN_pred_' + file_name[:-3] + 'pkl'))
predictions3 = pd.read_pickle(os.path.join(dir_, folder, group_info, '3_topN_pred_' + file_name[:-3] + 'pkl'))
predictions4 = pd.read_pickle(os.path.join(dir_, folder, group_info, '4_topN_pred_' + file_name[:-3] + 'pkl'))
predictions5 = pd.read_pickle(os.path.join(dir_, folder, group_info, '5_topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [5]:
predictions2[:5]

,uid,tid,rating
0,0,2,0.075900
1,0,14,0.026034
2,0,15,0.155446
3,0,20,0.042983
4,0,21,0.026271


In [6]:
predictions2['gid'] = predictions2['uid'].map(group2)
predictions3['gid'] = predictions3['uid'].map(group3)
predictions4['gid'] = predictions4['uid'].map(group4)
predictions5['gid'] = predictions5['uid'].map(group5)

In [7]:
predictions5[:5]

,uid,tid,rating,gid
0,0,2,0.075805,21.0
1,0,14,0.026464,21.0
2,0,15,0.155870,21.0
3,0,20,0.043121,21.0
4,0,21,0.026536,21.0


In [8]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,count,tags,rating,pop
7,0,2,4,0,3.00,24
24,0,14,2,1,1.50,6
25,0,15,12,0,4.75,53
33,0,20,3,12,2.50,12
34,0,21,1,5,0.25,6


In [9]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
len(groups_n[0][0]), len(groups_n[1][0]), len(groups_n[2][0]), len(groups_n[3][0])

(2, 3, 4, 5)

In [10]:
top_4_results = []
top_20_results = []
top_100_results = []

In [11]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):
    
    top_n_lists = []

    if len(groups[0]) == 2:
        predictions = predictions2
    elif len(groups[0]) == 3:
        predictions = predictions3
    elif len(groups[0]) == 4:
        predictions = predictions4
    else:
        predictions = predictions5

    for gid in range(len(groups)):
        
        rating_table = predictions[predictions['gid']==gid]
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        print(rating_table)
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

         uid    tid    rating  gid
5444849  144  30292  0.244456  0.0
5449373  144  34879  0.227945  0.0
5445312  144  30759  0.224795  0.0
5444815  144  30257  0.224720  0.0
5446639  144  32139  0.224654  0.0
...      ...    ...       ...  ...
5445794  144  31259  0.167065  0.0
5415712  144    823  0.167015  0.0
5526476  147   1117  0.166997  0.0
5453429  144  39008  0.166985  0.0
5446691  144  32191  0.166921  0.0

[100 rows x 4 columns]
         uid    tid    rating  gid
1675508   44  22628  0.244790  1.0
1673363   44  20466  0.243989  1.0
1672537   44  19626  0.236594  1.0
1675479   44  22597  0.225031  1.0
1673706   44  20810  0.222834  1.0
...      ...    ...       ...  ...
1675578   44  22698  0.158662  1.0
1677973   44  25108  0.158551  1.0
1672479   44  19567  0.158444  1.0
1681304   44  28470  0.158360  1.0
1675537   44  22657  0.156447  1.0

[100 rows x 4 columns]
         uid    tid    rating  gid
7007870  187      0  0.002287  2.0
7422081  198  15524  0.002287  2.0
7422087

         uid    tid    rating   gid
2897098   77  16303  0.257885  26.0
2902707   77  22066  0.252677  26.0
2897572   77  16791  0.247334  26.0
2898372   77  17624  0.246967  26.0
2897091   77  16296  0.243922  26.0
...      ...    ...       ...   ...
2902788   77  22149  0.175471  26.0
2917680   77  37453  0.175442  26.0
2902049   77  21393  0.175187  26.0
2900324   77  19638  0.174998  26.0
2900262   77  19573  0.174855  26.0

[100 rows x 4 columns]
         uid    tid    rating   gid
5073854  135   1556  0.258342  27.0
477899    12  34701  0.257820  27.0
445581    12     98  0.252587  27.0
5074880  135   2593  0.247407  27.0
5106213  135  34326  0.247347  27.0
...      ...    ...       ...   ...
478751    12  35553  0.197349  27.0
448691    12   3577  0.197170  27.0
5078301  135   6049  0.197067  27.0
5075121  135   2836  0.195242  27.0
5077100  135   4838  0.194946  27.0

[100 rows x 4 columns]
         uid    tid    rating   gid
1568037   41  28627  0.257942  28.0
3639189   96  37

         uid    tid    rating   gid
5035318  134   1209  0.257604  51.0
5044164  134  10389  0.257596  51.0
931035    24  36551  0.252454  51.0
5041302  134   7449  0.252400  51.0
5035807  134   1736  0.247062  51.0
...      ...    ...       ...   ...
932241    24  37757  0.196878  51.0
5042516  134   8701  0.196827  51.0
930775    24  36291  0.196527  51.0
5041041  134   7178  0.196476  51.0
930032    24  35548  0.194794  51.0

[100 rows x 4 columns]
         uid    tid    rating   gid
6892709  184      0  0.002287  52.0
7913739  211  13065  0.002287  52.0
7913737  211  13063  0.002287  52.0
7913736  211  13062  0.002287  52.0
7913735  211  13061  0.002287  52.0
...      ...    ...       ...   ...
7913824  211  13152  0.002287  52.0
7913830  211  13158  0.002287  52.0
7913829  211  13157  0.002287  52.0
7913828  211  13156  0.002287  52.0
7913827  211  13155  0.002287  52.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3280882   87  20592  0.258291  53.0
3273823   87  13

         uid    tid    rating   gid
2784497   74  16316  0.258200  77.0
2792620   74  25278  0.258192  77.0
2784923   74  16790  0.247637  77.0
2785649   74  17605  0.247270  77.0
2787842   74  19972  0.244738  77.0
...      ...    ...       ...   ...
2787376   74  19465  0.167668  77.0
2793521   74  26262  0.167592  77.0
2787340   74  19429  0.167459  77.0
2785857   74  17832  0.167440  77.0
2792988   74  25684  0.167358  77.0

[100 rows x 4 columns]
         uid    tid    rating   gid
4150361  110  19637  0.257944  78.0
4269323  113  25104  0.257751  78.0
4156254  110  25934  0.252735  78.0
4266400  113  22133  0.252554  78.0
4150830  110  20170  0.247390  78.0
...      ...    ...       ...   ...
4269568  113  25356  0.197143  78.0
4157460  110  27213  0.197102  78.0
4266136  113  21862  0.196605  78.0
4151906  110  21293  0.194928  78.0
4265389  113  21104  0.194872  78.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3084209   82  17704  0.257268  79.0
3093018   82  26

         uid    tid    rating   gid
2557638   68   3471  0.257966  17.0
2554723   68    268  0.252524  17.0
5151085  137   2402  0.247219  17.0
5151848  137   3205  0.247205  17.0
5150614  137   1899  0.243909  17.0
...      ...    ...       ...   ...
2558880   68   4800  0.177849  17.0
5154798  137   6248  0.177691  17.0
2588592   68  36274  0.177471  17.0
2558250   68   4124  0.177456  17.0
2588590   68  36272  0.177187  17.0

[100 rows x 4 columns]
         uid    tid    rating   gid
1956027   52   1716  0.258319  18.0
1984168   52  31703  0.258029  18.0
4161253  110  31113  0.257803  18.0
1989777   52  37619  0.252789  18.0
4166949  110  37051  0.252568  18.0
...      ...    ...       ...   ...
4163492  110  33428  0.189292  18.0
1985778   52  33370  0.189225  18.0
1956723   52   2470  0.189139  18.0
4163580  110  33521  0.189130  18.0
1957348   52   3152  0.189086  18.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3655830   97  15234  0.258242  19.0
3647161   97   6

         uid    tid    rating   gid
4274105  113  29916  0.252927  39.0
4269032  113  24813  0.247380  39.0
4088220  108  33326  0.247367  39.0
4088946  108  34054  0.247353  39.0
4091120  108  36235  0.244684  39.0
...      ...    ...       ...   ...
4091851  108  36967  0.189403  39.0
4090093  108  35207  0.189403  39.0
4270373  113  26170  0.189337  39.0
4057108  108   1683  0.189279  39.0
4277557  113  33416  0.189187  39.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3722625   99   4772  0.257659  40.0
3728631   99  11208  0.252426  40.0
3723115   99   5292  0.246888  40.0
3723934   99   6163  0.246874  40.0
3726310   99   8734  0.244209  40.0
...      ...    ...       ...   ...
3722923   99   5088  0.183091  40.0
3726298   99   8721  0.182988  40.0
3727844   99  10373  0.180956  40.0
3732834   99  15512  0.180683  40.0
3732154   99  14823  0.180636  40.0

[100 rows x 4 columns]
         uid  tid    rating   gid
5969025  159    0  0.002288  41.0
7751853  207  817  0

         uid    tid    rating   gid
1067652   28  18015  0.257917  13.0
1064726   28  15007  0.252690  13.0
1059400   28   9535  0.246718  13.0
144328     3  32207  0.244341  13.0
145128     3  33007  0.244327  13.0
...      ...    ...       ...   ...
1063247   28  13491  0.187927  13.0
1061292   28  11494  0.187920  13.0
1063439   28  13690  0.187593  13.0
144512     3  32391  0.186485  13.0
144366     3  32245  0.186482  13.0

[100 rows x 4 columns]
         uid    tid    rating   gid
1417253   37  26376  0.258499  14.0
259845     6  38388  0.258260  14.0
252107     6  30650  0.247047  14.0
4050972  107  34534  0.246876  14.0
252823     6  31366  0.246361  14.0
...      ...    ...       ...   ...
4051735  107  35325  0.191670  14.0
1417325   37  26448  0.191378  14.0
1418881   37  28011  0.191276  14.0
256408     6  34951  0.190834  14.0
4055433  107  39100  0.190697  14.0

[100 rows x 4 columns]
         uid    tid    rating   gid
3901590  103  34927  0.258530  15.0
509977    13  30

         uid    tid    rating  gid
3317479   88  20623  0.258457  3.0
2961702   79   4509  0.258384  3.0
2887982   77   6926  0.258073  3.0
2970022   79  13694  0.257578  3.0
2967279   79  10658  0.252910  3.0
...      ...    ...       ...  ...
3318160   88  21323  0.207910  3.0
2962519   79   5421  0.207850  3.0
2888853   77   7819  0.207596  3.0
2962835   79   5775  0.205949  3.0
3323578   88  27191  0.205903  3.0

[100 rows x 4 columns]
         uid    tid    rating  gid
3607519   96   4945  0.258305  4.0
1567214   41  27708  0.258147  4.0
2492010   66  12283  0.257696  4.0
3615467   96  13129  0.257499  4.0
2500904   66  22496  0.256891  4.0
...      ...    ...       ...  ...
2492560   66  12916  0.210558  4.0
3610518   96   8057  0.208548  4.0
184062     4  37508  0.208537  4.0
1569929   41  30660  0.208415  4.0
1568716   41  29360  0.208298  4.0

[100 rows x 4 columns]
         uid    tid    rating  gid
473142    12  29944  0.258120  5.0
2727897   72  30395  0.257676  5.0
2690340

In [12]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.05421686746987952, 0.0851063829787234, 0.13793103448275862, 0.14772727272727273]
------------------
[0.06445783132530121, 0.07872340425531915, 0.16724137931034483, 0.1318181818181818]
------------------
[0.06650602409638554, 0.07148936170212766, 0.17, 0.13272727272727272]


In [13]:
top_n_size = 100
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([100, 20, 4]):
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    if top_n_size == 4:
        top_4_results.append(avg_precisions)
    elif top_n_size == 20:
        top_20_results.append(avg_precisions)
    elif top_n_size == 100:
        top_100_results.append(avg_precisions)

In [14]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.05421686746987952, 0.0851063829787234, 0.13793103448275862, 0.14772727272727273]
[0.018072289156626505, 0.03723404255319149, 0.05172413793103448, 0.06818181818181818]
------------------
[0.06445783132530121, 0.07872340425531915, 0.16724137931034483, 0.1318181818181818]
[0.02590361445783133, 0.03829787234042553, 0.05344827586206898, 0.07954545454545457]
------------------
[0.06650602409638554, 0.07148936170212766, 0.17, 0.13272727272727272]
[0.02530120481927712, 0.04148936170212768, 0.0513793103448276, 0.06727272727272729]
